## Federal Workforce - Analyze Geospatial Distribution

### Import libraries

In [1]:
import geopandas as gpd
import requests
import json
import pandas as pd
import os 
import folium
from fredapi import fred
from tqdm import tqdm


import branca
from folium.plugins import TimeSliderChoropleth


## Set Dirs

In [2]:
# get root dir which ends in repo_name
repo_name = 'Fed_IT_Employment'
root = os.getcwd()
while os.path.basename(root) != repo_name:
    root = os.path.dirname(root)

# Get raw data directory
rdir = os.path.join( root, 'data', 'raw_data')
pdir = os.path.join( root, 'data', 'processed_data')

print(f"Base directory: {root}\nRaw data directory: {rdir}")

Base directory: /Users/coltonlapp/Dropbox/My Mac (Coltons-MacBook-Pro.local)/Desktop/Work/USDC/publicwork/Fed_IT_Employment
Raw data directory: /Users/coltonlapp/Dropbox/My Mac (Coltons-MacBook-Pro.local)/Desktop/Work/USDC/publicwork/Fed_IT_Employment/data/raw_data


## Load API keys

In [3]:
# Function to load API keys from the JSON file
def load_api_keys(filepath=os.path.join(root, 'api_keys.json')):
    with open(filepath) as f:
        keys = json.load(f)
    return keys

# Load the API keys
api_keys = load_api_keys()

# Access the Census API key
CENSUS_API_KEY = api_keys.get('census_api_key')
if CENSUS_API_KEY is None:
    raise ValueError('The Census API key is missing. Please provide a valid key.')
else:  
    print('Census API key loaded successfully.')


Census API key loaded successfully.


## Read in FedScope data

In [4]:
# Read in processed FedScope data
emp_fedscope = pd.read_csv(os.path.join(root, 'data', 'processed_data', 'FedScope_emp_data_concat.csv'),
                            parse_dates=['date'], 
                            usecols=['location', 'date', 'occupation']  )

# Convert date to datetime
emp_fedscope['date'] = pd.to_datetime(emp_fedscope['date'])

# Keep only columns with month is september 
emp_fedscope = emp_fedscope[emp_fedscope['date'].dt.month == 9]

emp_fedscope.head()

/var/folders/np/9tdxr2056n39qzb1mpdsjrtc0000gn/T/ipykernel_13487/836869369.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  emp_fedscope = pd.read_csv(os.path.join(root, 'data', 'processed_data', 'FedScope_emp_data_concat.csv'),


,location,occupation,date
59162895,IT-ITALY,1630,1998-09-01
59162896,RP-PHILIPPINES,1630,1998-09-01
59162897,BE-BELGIUM,1630,1998-09-01
59162898,FR-FRANCE,0810,1998-09-01
59162899,NL-NETHERLANDS,1630,1998-09-01


## Extract State FIPS from FedScope data

In [5]:
# Create "state_fips" column from "location". If start of column is digit, extract digit to new column 
emp_fedscope['state_fips'] = emp_fedscope['location'].str.extract(r'^(\d+)')

# Replace NA values with a default value (-1) and then convert to integer
emp_fedscope['state_fips'] = emp_fedscope['state_fips'].fillna(-1).astype(int)

emp_fedscope.head()

,location,occupation,date,state_fips
59162895,IT-ITALY,1630,1998-09-01,-1
59162896,RP-PHILIPPINES,1630,1998-09-01,-1
59162897,BE-BELGIUM,1630,1998-09-01,-1
59162898,FR-FRANCE,0810,1998-09-01,-1
59162899,NL-NETHERLANDS,1630,1998-09-01,-1


In [6]:
# Print summary information about number of rows with missing state_fips
print(
f"""Number of rows with missing state_fips: {(emp_fedscope['state_fips']==-1).sum()}
out of {emp_fedscope.shape[0]}, which is {((emp_fedscope['state_fips']==-1).sum())/emp_fedscope.shape[0]:.2%}
of the total rows."""
)

Number of rows with missing state_fips: 5599018
out of 51987859, which is 10.77%
of the total rows.


## Calculate number of employees in each state for each date

In [7]:
# Drop rows where location is NA
emp_fedscope = emp_fedscope[emp_fedscope['state_fips']!=-1]

job_series_groups = {'all': None,
                     'IT_managers_only': ['2210'],
                     'IT_DS_CS' : ['2210', '1550', '1560'], 
                     'IT_DS_CS_OR_MATH': ['2210', '1550', '1560', '1515', '1520']
                     }

# for each job group, group by state and date, counting the number of rows (i.e., employees) in that job group
for job_group, series in job_series_groups.items():
    if job_group == 'all':
        # Group by 'state_fips' and 'date' and count the number of rows
        emp_fedscope_grouped = emp_fedscope.groupby(['state_fips', 'date']).size().reset_index(name='total_employees_all')
    else:
        # Filter by job series, group by 'state_fips' and 'date', and count the number of rows
        emp_fedscope_grouped = emp_fedscope[emp_fedscope['occupation'].isin(series)].groupby(['state_fips', 'date']).size().reset_index(name=f'total_employees_{job_group}')

    # Merge results for all job groups
    if job_group == 'all':
        emp_fedscope_grouped_all = emp_fedscope_grouped
    else:
        emp_fedscope_grouped_all = pd.merge(emp_fedscope_grouped_all, emp_fedscope_grouped, on=['state_fips', 'date'], how='outer')



In [8]:
emp_fedscope_grouped_all.sample(10)

,state_fips,date,total_employees_all,total_employees_IT_managers_only,total_employees_IT_DS_CS,total_employees_IT_DS_CS_OR_MATH
326,16,2012-09-01,9653,240.0,240.0,241.0
1199,51,2001-09-01,109511,20.0,1060.0,2226.0
167,9,2009-09-01,8520,124.0,125.0,136.0
399,19,2007-09-01,8558,141.0,141.0,141.0
8,1,2006-09-01,36064,800.0,829.0,919.0
116,6,2010-09-01,172547,3668.0,4551.0,4920.0
1215,51,2017-09-01,133189,3706.0,3944.0,4529.0
19,1,2017-09-01,38129,723.0,762.0,922.0
119,6,2013-09-01,140110,2221.0,2630.0,2844.0
901,38,2015-09-01,5487,91.0,91.0,91.0


## Read in state population and shapefiles

In [9]:
states_shapes = gpd.read_file(os.path.join(pdir, 'states_shapefile'))
states_shapes.head()

,state,fips,geometry
0,Alabama,01,"POLYGON ((-87.359 35.001, -85.607 34.985, -85...."
1,Alaska,02,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2..."
2,Arizona,04,"POLYGON ((-109.043 37.000, -109.048 31.332, -1..."
3,Arkansas,05,"POLYGON ((-94.474 36.502, -90.153 36.496, -90...."
4,California,06,"POLYGON ((-123.233 42.006, -122.379 42.012, -1..."


In [10]:
state_population = pd.read_csv(os.path.join(pdir, 'state_population_2000_2023.csv'))
state_population.head()

,date,population,state_abbr,fips
0,2000-01-01,4452.173,AL,1
1,2001-01-01,4467.634,AL,1
2,2002-01-01,4480.089,AL,1
3,2003-01-01,4503.491,AL,1
4,2004-01-01,4530.729,AL,1


# Merge Datasets

In [13]:
# Convert fips columns to int
state_population['fips'] = state_population['fips'].astype(int)
states_shapes['fips'] = states_shapes['fips'].astype(int)

# Merge the population data with the states GeoDataFrame
states_gdf = states_shapes.merge(state_population, on='fips')

# Convert the 'date' column to datetime
states_gdf['date'] = pd.to_datetime(states_gdf['date'])

# Extract year from date column 
states_gdf['year'] = states_gdf['date'].dt.year


# Rename date to be df specific 
states_gdf = states_gdf.rename(columns={'date': 'population_date'})
emp_fedscope_grouped_all = emp_fedscope_grouped_all.rename(columns={'date': 'employment_data_date'})

# extract year from date column emp_fedscope_grouped_all
emp_fedscope_grouped_all['year'] = emp_fedscope_grouped_all['employment_data_date'].dt.year

# merge in the employment data
states_gdf = states_gdf.merge(emp_fedscope_grouped_all, left_on=['fips', 'year'], right_on=['state_fips', 'year'], how='left')


# employee_cols
employee_cols = ['total_employees_all', 'total_employees_IT_managers_only', 'total_employees_IT_DS_CS', 'total_employees_IT_DS_CS_OR_MATH']

# per capita employment
for col in employee_cols:
    states_gdf[f'{col}_per_100thousand'] = states_gdf[col] / states_gdf['population']

# Show the merged GeoDataFrame
states_gdf.head()

,state,fips,geometry,population_date,population,state_abbr,year,state_fips,employment_data_date,total_employees_all,total_employees_IT_managers_only,total_employees_IT_DS_CS,total_employees_IT_DS_CS_OR_MATH,total_employees_all_per_100thousand,total_employees_IT_managers_only_per_100thousand,total_employees_IT_DS_CS_per_100thousand,total_employees_IT_DS_CS_OR_MATH_per_100thousand
0,Alabama,1,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2000-01-01,4452.173,AL,2000,1,2000-09-01,33508,NaN,9.0,110.0,7.526212,NaN,0.002021,0.024707
1,Alabama,1,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2001-01-01,4467.634,AL,2001,1,2001-09-01,33636,NaN,18.0,210.0,7.528817,NaN,0.004029,0.047005
2,Alabama,1,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2002-01-01,4480.089,AL,2002,1,2002-09-01,33721,664.0,696.0,859.0,7.526859,0.148211,0.155354,0.191737
3,Alabama,1,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2003-01-01,4503.491,AL,2003,1,2003-09-01,33689,1103.0,1120.0,1282.0,7.480641,0.244921,0.248696,0.284668
4,Alabama,1,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",2004-01-01,4530.729,AL,2004,1,2004-09-01,35139,1099.0,1137.0,1306.0,7.755706,0.242566,0.250953,0.288254


## Visualize IT Managers Per Capita Across Time

In [14]:

# Check if the GeoDataFrame has a CRS
if states_gdf.crs is None:
    # Assign EPSG:4326 (WGS84) if no CRS is set
    states_gdf = states_gdf.set_crs(epsg=4326)

# Ensure the 'year' column is an integer
states_gdf['year'] = states_gdf['year'].astype(int)

# Sort the data by year
states_gdf = states_gdf.sort_values('year')

# Filter to keep rows with year > 2004
states_gdf = states_gdf[states_gdf['year'] > 2004]

# drop rows where total_employees_IT_managers_only is NA
states_gdf = states_gdf[states_gdf['total_employees_IT_managers_only_per_100thousand'].notna()]

# Ensure the 'year' column is an integer
states_gdf['year'] = states_gdf['year'].astype(int)

# Convert the 'year' into a datetime object representing January 1st of that year
states_gdf['date'] = pd.to_datetime(states_gdf['year'], format='%Y')

# Convert the 'date' column into Unix timestamps (seconds since epoch)
states_gdf['timestamp'] = states_gdf['date'].astype(int) // 10**9

# Create a color map using branca.colormap (for smooth gradient)
max_population = states_gdf['total_employees_IT_managers_only_per_100thousand'].max()
min_population = states_gdf['total_employees_IT_managers_only_per_100thousand'].min()
colormap = branca.colormap.linear.YlGnBu_09.scale(min_population, max_population)  # You can adjust color palette here

# Create a dictionary to store the population data for each feature at each timestamp
styledict = {}
for i, row in states_gdf.iterrows():
    fips = row['fips']
    timestamp = row['timestamp']  # This is now based on 'year'
    population = row['total_employees_IT_managers_only_per_100thousand']
    
    if fips not in styledict:
        styledict[fips] = {}
    
    # Use the colormap to assign colors based on population
    color = colormap(population)
    # black outline
    styledict[fips][timestamp] = {'color': color, 'opacity': 0.7, 'weight': 1, 'fillOpacity': 0.7, 'fillColor': color, 'lineColor': 'black'}

# Initialize the base map
m = folium.Map(location=[37, -102], zoom_start=5)

# Add TimeSliderChoropleth to the map
gjson = states_gdf.set_index('fips')['geometry'].__geo_interface__  # Get the GeoJSON interface for the geometries
TimeSliderChoropleth(
    gjson,
    styledict=styledict,
).add_to(m)

# Add a colormap legend to the map
colormap.caption = 'IT Managers Population (2210) Per Hundred Thousand People'
colormap.add_to(m)

# Add the title and subtitle to the map
title_html = '''
     <h3 align="center" style="font-size:20px"><b>Federal IT Workers (Series 2210) Per 100K people over time</b></h3>
     <h4 align="center" style="font-size:15px"><i>Data source: FedScope Employment Cube from OPM</i></h4>
     '''
m.get_root().html.add_child(folium.Element(title_html))

# save output to root/output/IT_workers_per_capita.html
m.save(os.path.join(root, 'output', 'FED_IT_workers_per_capita.html'))

# If in a Jupyter Notebook, use this to display:
m